In [1]:
import numpy as np
import os
from tensorflow import keras
import kerasncp as kncp
import matplotlib.pyplot as plt
import seaborn as sns

In [41]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = x_train.reshape((60000, 1, 28**2))
x_test = x_test.reshape((10000, 1, 28**2))

y_train = keras.utils.to_categorical(y_train).reshape(60000, 1, 10)
y_test = keras.utils.to_categorical(y_test).reshape(10000, 1, 10)

In [66]:
ncp_arch = kncp.wirings.NCP(
    inter_neurons=100,  # Number of inter neurons
    command_neurons=20,  # Number of command neurons
    motor_neurons=10,  # Number of motor neurons
    sensory_fanout=2,  # How many outgoing synapses has each sensory neuron
    inter_fanout=2,  # How many outgoing synapses has each inter neuron
    recurrent_command_synapses=3,  # Now many recurrent synapses are in the
    # command neuron layer
    motor_fanin=4,  # How many incomming syanpses has each motor neuron
)
ncp_cell = kncp.LTCCell(ncp_arch)

ncp_model = keras.models.Sequential(
    [
        keras.layers.Reshape((28, 28, 1), input_shape=(28, 28)),
        keras.layers.Conv2D(filters=16, kernel_size=3),
        keras.layers.MaxPool2D(3),
        keras.layers.Flatten(),
        keras.layers.Reshape((None, 100)),
        # keras.layers.InputLayer(input_shape=(None, 28**2)),
        keras.layers.RNN(ncp_cell, return_sequences=True),
    ]
)
ncp_model.compile(
    optimizer=keras.optimizers.Adam(0.01), loss='categorical_crossentropy', metrics=['accuracy']
)

ValueError: Tried to convert 'shape' to a tensor and failed. Error: None values not supported.

In [62]:
ncp_model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or calling `fit()` with some data, or specify an `input_shape` argument in the first layer(s) for automatic build.

In [53]:
ncp_model.fit(x_train, y_train)

 333/1875 [====>.........................] - ETA: 3:40 - loss: 8.9542 - accuracy: 0.1012

KeyboardInterrupt: 

In [37]:
ncp_model.predict(np.array([x_train[0]]))

array([[[0.09602951, 0.12941141, 0.07840873, 0.10468204, 0.09348159,
         0.09267961, 0.11822827, 0.11323945, 0.07945774, 0.09612259]]],
      dtype=float32)